# 使用NOVAS 完成上机作业


# Now, waiting to do:

## How to use Novas to calculate: some Earth Motivate Parameters?

> the Parameters has:
> 1. 自行 ——> Hippaku
> * 周年视差
> * 光线引力弯曲
> * 周年光行差
> * 岁差
> * 章动



## How it shows in the guidebook
* Proper motion (generalized): the three-dimensional space motion of the star, relative to that of the solar system barycenter, between the catalog epoch and the date of interest. Assumed linear and computed from the catalog proper motion components, radial velocity, and parallax. Projected onto the sky, the motion amounts to less than 1 arcsecond per year (usually much less) except for a few nearby stars. [starvectors, proper_motion]
  
* Parallax: the change in our perspective on stars in the solar neighborhood due to the position of the Earth in its orbit. Its magnitude is (distance in parsecs)-1 and, hence, is always less than 1 arcsecond. [bary2obs]
* Gravitational light bending: the apparent deflection of the light path in the gravitational field of the Sun and, to a much lesser extent, the other planets. Although it reaches 1.8 arcsecond at the limb of the Sun, it falls to 0.05 arcsecond 10o from the Sun and amounts to no more than a few milliarcseconds over the hemisphere of the sky opposite the Sun. [grav_def, grav_vec]
* Aberration: the change in the apparent direction of light caused by the observer’s velocity (v) with respect to the solar system barycenter. Independent of distance, it is approximately equal to v/c, expressed as an angle. Therefore, it can reach 21 arcseconds for observers on the surface of the Earth and somewhat more for instruments in orbit. [aberration]
* Atmospheric refraction: the total angular change in the direction of the light path through the Earth’s atmosphere; applies only to an observer on, or near, the surface of the Earth. The direction of refraction is always assumed to be parallel to the local vertical and a function only of zenith distance although these assumptions may not be true in all cases. At optical wavelengths, its magnitude is zero at the zenith, about 1 arcminute at a zenith distance of 45°, and 0.5° at the horizon. Refraction is roughly proportional to the atmospheric pressure at the observer, but it also depends on other atmospheric parameters and the observational wavelength. [equ2hor, refract]


In [1]:
import numpy as np
import pandas as pd

from novas import compat as novas
from novas.compat import eph_manager

import de405

In [2]:
# 使用skyfield包（注意需要科学上网下载deltat表）
from skyfield.api import load
import skyfield
import datetime

ts = skyfield.api.load.timescale()

[#################################] 100% deltat.preds


In [3]:
# Example for using Skyfield

t = ts.tdb(jd=2305424.5)
tt = t.utc_datetime()
print(tt)

1599-12-08 23:59:17.817000+00:00


In [4]:
eph_manager.ephem_open('/Users/dengdingshan/anaconda3/lib/python3.6/site-packages/novas_de405/DE405.bin')

(2305424.5, 2525008.5, 405)

# Exercise

## Exercise I
CDS上下载Bright Star Catalogue, 5th Revised Ed.星表，

利用其所给的恒星的位置，给出2018年６月１日24点（UTC），

在北京兴隆观测站利用郭守敬望远镜（原LAMOST）所能观测到的亮于7等的恒星的视位置、测站地平坐标，

要求位置精度在0.1arcsec以下。

*
注：星表自行下载；兴隆的地理坐标自行查找；郭守敬望远镜的观测特性自行搜索（视场等参数）；请根据精度要求取舍位置归算过程中的修正项。*




### 相关数据；星表整理

LAMOST安放在中国科学院国家天文台兴隆观测站，该站地处燕山主峰南麓，位于河北省兴隆县连营寨（东经7小时50分，北纬40度23分），海拔960米。

In [22]:
(7+50/60)/24 * 360

117.5

In [5]:
# Xinglong
# 海拔960米，地理坐标为东经117度34.5分，北纬40度23分36秒

Longitude = +(117 + 34.5/60) # degree
Latitude = +(40 + 23.36/60) # degree
Altitude = 960 # m

In [6]:
# date
# delta_t = tt - ut1
jd_tt = novas.julian_date(day=2,hour=0,month=6,year=2018)
Time = ts.tt(jd=jd_tt)
delta_t = Time.tt - Time.ut1 # days
delta_t = delta_t*(24*3600) # s

In [7]:
# Data_read
data_dir = '/Users/dengdingshan/Documents/data/BrightStar/BrightStar.csv'
BS_df = pd.read_csv(data_dir)
BS_df.loc[26]

HR          27.000000
DEJ2000     46.072200
RAJ2000      0.172028
Vmag         5.030000
pmRA         0.008000
pmDE         0.000000
Parallax    -0.002000
RadVel      -5.000000
Name: 26, dtype: float64

In [8]:
# only Vmag < 7 accepted
# if Plx is 0, we give it 0.001

BS_df_fil = BS_df[BS_df['Vmag'] < 7]
BS_df_fil = BS_df_fil[BS_df_fil['Parallax'] >= 0]
BS_df_fil.loc[:,'Format_Parallax'] = BS_df_fil.copy()['Parallax'].replace(0,0.001).values
BS_df_fil = BS_df_fil.reset_index()

In [9]:
BS_df_fil

,index,HR,DEJ2000,RAJ2000,Vmag,pmRA,pmDE,Parallax,RadVel,Format_Parallax
0,0,1,45.2292,0.086083,6.70,-0.012,-0.018,0.000,-18,0.001
1,1,2,-0.5031,0.084389,6.29,0.045,-0.060,0.000,14,0.001
2,2,3,-5.7075,0.088917,4.61,-0.009,0.089,0.014,-6,0.014
3,3,4,13.3961,0.095000,5.51,0.045,-0.012,0.000,-2,0.001
4,4,5,58.4367,0.104444,5.96,0.263,0.030,0.047,-12,0.047
5,5,6,-49.0750,0.105278,5.70,0.565,-0.038,0.050,3,0.050
6,6,7,64.1961,0.107361,5.59,0.008,0.000,0.000,0,0.001
7,7,8,29.0214,0.110222,6.13,0.380,-0.182,0.067,-8,0.067
8,8,9,-23.1075,0.113917,6.18,0.100,-0.045,0.000,3,0.001
9,9,10,-17.3864,0.121722,6.19,-0.018,0.036,0.000,-9,0.001


### 计算恒星的视位置和测站位置

In [10]:
# Parameter (OnSurface) for calculate Topo Position
Exercise_I_OnSurface = novas.make_on_surface(height=Altitude,\
                                             latitude=Latitude,\
                                             longitude=Longitude,\
                                             pressure=1010,\
                                             temperature=20)

jd_ut1 = Time.ut1
Epp_x = 0
Epp_y = 0

In [15]:
Exercise_I_app = novas.app_star(jd_tt,Exercise_I_Cat,accuracy)
Exercise_I_app

(0.10168405880118705, 45.32582078657546)

In [21]:
## app_star and topo_star

app_ra = []
app_dec = []
topo_dec = []
topo_ra = []

ZD = []
AZ = []
RAR = []
DECR = []

for i in range(len(BS_df_fil)):

    RA = BS_df_fil['RAJ2000'].values[i]
    DEC = BS_df_fil['DEJ2000'].values[i]
    pm_RA = BS_df_fil['pmRA'].values[i]
    pm_DEC = BS_df_fil['pmDE'].values[i]
    plx = BS_df_fil['Format_Parallax'].values[i]
    Vr = BS_df_fil['RadVel'].values[i]
    
    Exercise_I_Cat = novas.make_cat_entry('ExerciseI','HIP',0,RA,DEC,pm_RA,pm_DEC,plx,Vr)
 
    jd_tt = jd_tt

    accuracy = 0
    
    Exercise_I_app = novas.app_star(jd_tt,Exercise_I_Cat,accuracy)
    
    app_ra.append(Exercise_I_app[0])
    app_dec.append(Exercise_I_app[1])
    
    delta_t = delta_t
    
    RA_topo,DEC_topo = novas.topo_star(jd_tt, delta_t, Exercise_I_Cat, Exercise_I_OnSurface,accuracy)
    
    topo_ra.append(RA_topo); topo_dec.append(DEC_topo)
    
    
    jd_ut1 = Time.ut1
    
    (zd,az),(rar,decr) = \
              novas.equ2hor(jd_ut1=jd_ut1,\
              delta_t=delta_t,\
              xp=Epp_x ,yp=Epp_y ,\
              location=Exercise_I_OnSurface,\
              ra=RA_topo,dec=DEC_topo,\
              ref_option=1,accuracy=0)
    
    ZD.append(zd);AZ.append(az);RAR.append(rar);DECR.append(decr)
    
    
BS_df_right = pd.DataFrame({'appRA':app_ra,
                           'appDE':app_dec,
                           'topoRA':topo_ra,
                           'topoDE':topo_dec,
                           'zd':ZD,'az':AZ,'rar':RAR,'decr':DECR})
BS_df_new = pd.concat([BS_df_fil,BS_df_right],axis=1)
BS_df_new

,index,HR,DEJ2000,RAJ2000,Vmag,pmRA,pmDE,Parallax,RadVel,Format_Parallax,appDE,appRA,az,decr,rar,topoDE,topoRA,zd
0,0,1,45.2292,0.086083,6.70,-0.012,-0.018,0.000,-18,0.001,45.325821,0.101684,319.549966,45.324591,0.101806,45.325826,0.101690,6.691517
1,1,2,-0.5031,0.084389,6.29,0.045,-0.060,0.000,14,0.001,-0.402922,0.099754,189.445146,-0.389981,0.099867,-0.402922,0.099759,41.168785
2,2,3,-5.7075,0.088917,4.61,-0.009,0.089,0.014,-6,0.014,-5.606846,0.104255,188.455102,-5.591332,0.104377,-5.606847,0.104259,46.325461
3,3,4,13.3961,0.095000,5.51,0.045,-0.012,0.000,-2,0.001,13.495025,0.110434,192.847917,13.502642,0.110531,13.495026,0.110439,27.403294
4,4,5,58.4367,0.104444,5.96,0.263,0.030,0.047,-12,0.047,58.532694,0.120210,350.283560,58.527861,0.120376,58.532700,0.120219,18.521516
5,5,6,-49.0750,0.105278,5.70,0.565,-0.038,0.050,3,0.050,-48.970676,0.120216,183.866867,-48.591432,0.123223,-48.970681,0.120223,89.131176
6,6,7,64.1961,0.107361,5.59,0.008,0.000,0.000,0,0.001,64.291904,0.123228,353.793633,64.285348,0.123433,64.291910,0.123238,24.138144
7,7,8,29.0214,0.110222,6.13,0.380,-0.182,0.067,-8,0.067,29.119052,0.125750,204.683849,29.122058,0.125845,29.119055,0.125755,12.229743
8,8,9,-23.1075,0.113917,6.18,0.100,-0.045,0.000,3,0.001,-23.005288,0.129130,185.921391,-22.975529,0.129320,-23.005290,0.129135,63.591570
9,9,10,-17.3864,0.121722,6.19,-0.018,0.036,0.000,-9,0.001,-17.284715,0.136973,186.364722,-17.261112,0.137124,-17.284717,0.136978,57.889177


In [18]:
# jd_ut1 = Time.ut1

# (zd,az),(rar,decr) = \
#               novas.equ2hor(jd_ut1=jd_ut1,\
#               delta_t=delta_t,\
#               xp=Epp_x ,yp=Epp_y ,\
#               location=Exercise_I_OnSurface,\
#               ra=RA_topo,dec=DEC_topo,\
#               ref_option=1,accuracy=0)


# (zd, az) : tuple of floats
#     Topocentric (zenith distance, azimuth) in degrees. Zenith
#     distance is affected by refraction if 'ref_option', is
#     non-zero. Azimuth is measured east from north.
# (rar, decr) : tuple of floats
#     Topocentric (right ascension in hours, declination in
#     degrees) of object of interest, referred to true equator and
#     equinox of date, affected by refraction if 'ref_option' is
#     non-zero.


### 选取LAMOST可以观测的源

时角正负两小时，赤纬-10度到90度



## Exercise II

计算2018年６月１日24点（UTC），

月球、金星、火星、木星和土星的站心位置、地平坐标，要求位置精度在0.1arcsec以下。

请结合第一个实验中得到的恒星位置，说明此时的月球、金星、火星、木星和土星是否会影响恒星的观测。

*
注：请根据精度要求取舍位置归算过程中的修正项。*


## Tips

程序中涉及到的时间转化，请同步完成。由于只计算一个时刻的位置，可以简化相关数据文件读取过程。

*
注：相关数据文件请到IERS网站查找。*
